In [ ]:
import numpy as np
from scipy.optimize import fsolve
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"
# marker style
marker01=dict(size=5, symbol='x-thin', line=dict(width=1, color='rgb(76,114,176)'))
marker02=dict(size=5, symbol='x-thin', line=dict(width=1, color='rgb(221,132,82)'))
marker03=dict(size=5, symbol='x-thin', line=dict(width=1, color='rgb(85,168,104)'))

# Bead on a hoop

A circular wire hoop rotates with constant angular velocity $\omega$ about a vertical diameter. A small bead
moves, without friction, along the hoop.

The equation of motion can be shown to be (using the standard notation in classical mechanics) :

$$ \ddot{\theta} = -\omega_c^2 \sin \theta + \omega^2 \sin \theta \, \cos \theta $$

with $\omega_c = \sqrt{g/R}$, where the gravity acceleration is denoted by ($g=9.81$) and the radius of the hoop is denoted $R$ ($R=1$). 

Let $q=\theta$ and $p = \dot{\theta}$. Then, we can switch to a first order system of differential equations :

$$
\left\{\begin{aligned}
\dot{p} & =  -(\omega_c^2 - \omega^2 \cos q) \sin q \\
\dot{q} & = p
\end{aligned}\right.
$$

In [ ]:
class bead_hoop_model:

    def __init__(self, omega):
        self.omega = omega

    def fcn(self, t, y):
        q, p = y
        q_dot = self.H_p(p)
        p_dot = -self.H_q(q)
        return np.array([q_dot, p_dot])

    def H_p(self, p):
        return p
    
    def H_q(self, q):
        omega = self.omega
        omega_c = np.sqrt(9.81)
        return -np.sin(q)*(omega*omega*np.cos(q) - omega_c*omega_c)

    def hamiltonian(self, y):
        q, p = np.split(y,2)
        omega = self.omega 
        oo = omega*omega
        return 0.5*(-oo*np.sin(q)*np.sin(q) + p*p) - 9.81*np.cos(q)

In [ ]:
#################################################################
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

#################################################################
def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*fcn(tn, yn)

    return ode_result(y, t)

#################################################################
def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*fcn(tip1, uip1)

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*fcn(tn, yn)
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        y[:,it+1] = fsolve(g, y0, (yn, tn+dt))

    return ode_result(y, t)

#################################################################
def symplectic_euler(tini, tend, nt, yini, H_p, H_q):    
    # This version only works for separable Hamiltionians (H(p,q)=T(p)+U(q)),
    # which allows us to remove the implicit terms

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    q, p = np.split(y, 2)
    for it in range(nt-1):
        q_n = q[:,it]
        p_n = p[:,it]
        q[:,it+1] = q_n + dt*H_p(p_n)
        p[:,it+1] = p_n - dt*H_q(q[:,it+1])

    return ode_result(y,t)

## Fisrt order schemes

In [ ]:
qini = 0.
pini = 1.
yini = (qini, pini)
    
tini = 0.
tend = 10.
nt = 1001

bhm = bead_hoop_model(omega=1)
H_p = bhm.H_p
H_q = bhm.H_q
fcn = bhm.fcn
hamiltonian = bhm.hamiltonian

# forward Euler integration
sol_fe = forward_euler(tini, tend, nt, yini, fcn)
ham_fe = hamiltonian(sol_fe.y)

# backward Euler integration
sol_be = backward_euler(tini, tend, nt, yini, fcn)
ham_be = hamiltonian(sol_be.y)

# symplectic Euler integration
sol_se = symplectic_euler(tini, tend, nt, yini, H_p, H_q)
ham_se = hamiltonian(sol_se.y)

# plot phase plan
fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=sol_fe.y[0], y=sol_fe.y[1], mode='markers', marker=marker01, name='forward euler'))
fig_sol.add_trace(go.Scatter(x=sol_be.y[0], y=sol_be.y[1], mode='markers', marker=marker02, name='backward euler'))
fig_sol.add_trace(go.Scatter(x=sol_se.y[0], y=sol_se.y[1], mode='markers', marker=marker03, name='symplectic euler'))
fig_sol.update_layout(title='Solution in phase space', xaxis_title='p', yaxis_title='q', margin=dict(b=0))
fig_sol.show()

# plot Hamiltonian
fig_ham = go.Figure()
fig_ham.add_trace(go.Scatter(x=sol_fe.t, y=ham_fe[0], mode='markers', marker=marker01, name='forward euler'))
fig_ham.add_trace(go.Scatter(x=sol_be.t, y=ham_be[0], mode='markers', marker=marker02, name='backward euler'))
fig_ham.add_trace(go.Scatter(x=sol_se.t, y=ham_se[0], mode='markers', marker=marker03, name='symplectic euler'))
fig_ham.update_layout(title='Hamiltonian', xaxis_title='t', yaxis_title='Hamiltonian')
fig_ham.show()

## Symplectic schemes

In [ ]:
#################################################################
def stormer_verlet(tini, tend, nt, yini, H_p, H_q):
    
    dt = (tend-tini) / (nt-1) 
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array
    
    q, p = np.split(y, 2)
    for it in range(nt-1):
        q_n = q[:,it]
        p_n = p[:,it]
        p_np05 = p_n - (dt/2)*H_q(q_n)
        q[:,it+1] = q_n + dt*H_p(p_np05)
        p[:,it+1] = p_np05 - (dt/2)*H_q(q[:,it+1])
        
    return ode_result(y, t)

#################################################################
def optimized_815(tini, tend, nt, yini, H_p, H_q):
    
    def stormer_verlet_step(dt, yini, H_p, H_q):
        q_n, p_n = np.split(yini,2)
        p_np05 = p_n - (dt/2)*H_q(q_n)
        q_np1  = q_n + dt*H_p(p_np05)
        p_np1  = p_np05 - (dt/2)*H_q(q_np1)
        return np.concatenate((q_np1, p_np1))
     
    dt = (tend-tini) / (nt-1) 
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    nstep = 15
    gamma = np.zeros(nstep+1)
    gamma[0]  =  0.
    gamma[1]  =  0.74167036435061295344822780
    gamma[2]  = -0.40910082580003159399730010
    gamma[3]  =  0.19075471029623837995387626
    gamma[4]  = -0.57386247111608226665638773
    gamma[5]  =  0.29906418130365592384446354
    gamma[6]  =  0.33462491824529818378495798
    gamma[7]  =  0.31529309239676659663205666
    gamma[8]  = -0.79688793935291635401978884
    gamma[9]  = gamma[7]
    gamma[10] = gamma[6]
    gamma[11] = gamma[5]
    gamma[12] = gamma[4]
    gamma[13] = gamma[3]
    gamma[14] = gamma[2]
    gamma[15] = gamma[1]
                          
    ytmp = y[:,0]

    for it in range(nt-1):
 
        for istep in range(nstep):

            ti = 0.0
            te = gamma[istep+1]*dt
            ytmp = stormer_verlet_step(te-ti, ytmp, H_p, H_q)

        y[:,it+1] = ytmp
        
    return ode_result(y, t)

In [ ]:
qini = 0.
pini = 1.
yini = (qini, pini)
    
tini = 0.
tend = 10.
nt = 101

bhm = bead_hoop_model(omega=1)
H_p = bhm.H_p
H_q = bhm.H_q
fcn = bhm.fcn
hamiltonian = bhm.hamiltonian

# symplectic Euler integration
sol_se = symplectic_euler(tini, tend, nt, yini, H_p, H_q)
ham_se = hamiltonian(sol_se.y)

# Stormer Verlet integration
sol_ve = stormer_verlet(tini, tend, nt, yini, H_p, H_q)
ham_ve = hamiltonian(sol_ve.y)

# optimized order 8 step 15 integration 
sol_o815 = optimized_815(tini, tend, nt, yini, H_p, H_q)
ham_o815 = hamiltonian(sol_o815.y)

# plot phase plan
fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=sol_se.y[0], y=sol_se.y[1], mode='markers', marker=marker01, name='symplectic euler'))
fig_sol.add_trace(go.Scatter(x=sol_ve.y[0], y=sol_ve.y[1], mode='markers', marker=marker02, name='Stormer-Verlet'))
fig_sol.add_trace(go.Scatter(x=sol_o815.y[0], y=sol_o815.y[1], mode='markers', marker=marker03, name='optimized 815'))
fig_sol.update_layout(title='Solution in phase space', xaxis_title='p', yaxis_title='q', margin=dict(b=0))
fig_sol.show()

# plot Hamiltonian
fig_ham = go.Figure()
fig_ham.add_trace(go.Scatter(x=sol_se.t, y=ham_se[0], mode='markers', marker=marker01, name='symplectic euler'))
fig_ham.add_trace(go.Scatter(x=sol_ve.t, y=ham_ve[0], mode='markers', marker=marker02, name='Stormer-Verlet'))
fig_ham.add_trace(go.Scatter(x=sol_o815.t, y=ham_o815[0], mode='markers', marker=marker03, name='optimized 815'))
fig_ham.update_layout(title='Hamiltonian', xaxis_title='t', yaxis_title='Hamiltonian')
fig_ham.show()

## Composition scheme optimized 8-15 vs Dopri853

In [ ]:
qini = 0.
pini = 1.
yini = (qini, pini)
    
tini = 0.
tend = 10.
nt = 101

bhm = bead_hoop_model(omega=1)
H_p = bhm.H_p
H_q = bhm.H_q
fcn = bhm.fcn
hamiltonian = bhm.hamiltonian

# optimized order 8 step 15 integration
sol_o815 = optimized_815(tini, tend, nt, yini, H_p, H_q)
ham_o815 = hamiltonian(sol_o815.y)
print(f"Composition scheme optimized 8-15 accepted steps : ", nt)

# Dopri 853 integration
tol = 1e-6
sol_dop853 = solve_ivp(fcn, (tini, tend), yini, method='DOP853', atol=tol, rtol=tol)
ham_dop853 = hamiltonian(sol_dop853.y)
print(f"Dopri 853 accepted steps : ", len(sol_dop853.t))

# plot phase plan
fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=sol_o815.y[0], y=sol_o815.y[1], mode='markers', marker=marker01, name='optimized 815'))
fig_sol.add_trace(go.Scatter(x=sol_dop853.y[0], y=sol_dop853.y[1], mode='markers', marker=marker02, name='Dopri 853'))
fig_sol.update_layout(title='Solution in phase space', xaxis_title='p', yaxis_title='q', margin=dict(b=0))
fig_sol.show()

# plot Hamiltonian
fig_ham = go.Figure()
fig_ham.add_trace(go.Scatter(x=sol_o815.t, y=ham_o815[0], mode='markers', marker=marker01, name='optimized 815'))
fig_ham.add_trace(go.Scatter(x=sol_dop853.t, y=ham_dop853[0], mode='markers', marker=marker02, name='Dopri 853'))
fig_ham.update_layout(title='Hamiltonian', xaxis_title='t', yaxis_title='Hamiltonian')
fig_ham.show()